Train gradient boosting models on the data.

# 1. Load data and packages

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from handyspark import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import LogisticRegression

In [4]:
%run Users/tw2567@columbia.edu/Udacity/churn_prediction/util

In [5]:
raw_data = spark.read.parquet('s3://tianyi-wang-data-science-projects/churn-prediction-2020/features1')\
           .withColumnRenamed('last_status_cancelled','label')\
           .drop('userId')
features_col = [i for i in raw_data.columns if i != 'label']
assembler = VectorAssembler(inputCols=features_col,outputCol="features")
data = assembler.transform(raw_data).select('features','label')

In [6]:
feature_names = assembler.getInputCols()

In [7]:
display(data.groupby('label').count())

label,count
1,5005
0,17272


# 2 Check whether it's better to upsample the positive group

### 2.1 Triple the size of positive samples

In [10]:
auc = tune_sampling_parameters(data, LogisticRegression(), 3, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 10510.5
Majority -- training size went from 12090.4 to 12090.4
fold 0
10512
11938
fold 1
10337
11968
fold 2
10432
11975
Out[6]: [0.7654479156400906, 0.7656033768393352, 0.764435777426504]

### 2.2 Double the size of positive samples

In [12]:
auc = tune_sampling_parameters(data, LogisticRegression(), 2, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 7007.0
Majority -- training size went from 12090.4 to 12090.4
fold 0
7101
11966
fold 1
7040
11970
fold 2
7004
11963
Out[9]: [0.7653214772296661, 0.7629500630109982, 0.7643189291549938]

### 2.3 Postive group size * 1.5

In [14]:
auc = tune_sampling_parameters(data, LogisticRegression(), 1.5, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 5255.25
Majority -- training size went from 12090.4 to 12090.4
fold 0
5329
11977
fold 1
5340
11974
fold 2
5281
11975
Out[10]: [0.7635282474387072, 0.7619870111663289, 0.7636734875688457]

### 2.4 Don't upsample

In [16]:
auc = tune_sampling_parameters(data, LogisticRegression(), 1, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 3503.5
Majority -- training size went from 12090.4 to 12090.4
fold 0
3405
11957
fold 1
3534
11972
fold 2
3468
11988
Out[11]: [0.7624103653249854, 0.7607970768247102, 0.7631085526751629]